In [1]:
from collections import Counter
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import re
import pandas as pd
import numpy as np
import datetime as dt

## Load Databases

In [2]:
companies_df = pd.read_csv("/Users/jbpatty/project-3/companies.csv")

acquisitions_df = pd.read_csv("/Users/jbpatty/project-3/acquisitions.csv")

investments_df = pd.read_csv("/Users/jbpatty/project-3/investments.csv")

rounds_df = pd.read_csv("/Users/jbpatty/project-3/rounds.csv")

additions_df = pd.read_csv("/Users/jbpatty/project-3/additions.csv") 

## Clean Databases

### Company Database

In [3]:
companies_df.dtypes

permalink            object
name                 object
homepage_url         object
category_list        object
funding_total_usd    object
status               object
country_code         object
state_code           object
region               object
city                 object
funding_rounds        int64
founded_at           object
first_funding_at     object
last_funding_at      object
dtype: object

In [4]:
companies_df.shape

(66368, 14)

#### Remove companies that do not have a name

In [5]:
clean_companies_df = companies_df.dropna(subset = ['name'])
clean_companies_df.shape

(66367, 14)

#### Remove companies that do not have a first funding date

In [6]:
clean_companies_df = clean_companies_df.dropna(subset = ['first_funding_at'])
clean_companies_df.shape

(66343, 14)

#### Remove dates missing a year (year 1000 is just wrong) for both first and last funding date using raw string notation from Regex. Years will start with 19__ or 20__

In [7]:
clean_companies_df = clean_companies_df[clean_companies_df.first_funding_at.str.match(r'(19)|(20)')]

clean_companies_df = clean_companies_df[clean_companies_df.last_funding_at.str.match(r'(19)|(20)')]

clean_companies_df.shape

(66338, 14)

#### Adding datetime functionality 

In [8]:
clean_companies_df.first_funding_at = pd.to_datetime(clean_companies_df.loc[:, 'first_funding_at'])

clean_companies_df.last_funding_at = pd.to_datetime(clean_companies_df.loc[:, 'last_funding_at'])

clean_companies_df.dtypes

permalink                    object
name                         object
homepage_url                 object
category_list                object
funding_total_usd            object
status                       object
country_code                 object
state_code                   object
region                       object
city                         object
funding_rounds                int64
founded_at                   object
first_funding_at     datetime64[ns]
last_funding_at      datetime64[ns]
dtype: object

#### Convert funding total and funding rounds from objects to numeric values

In [9]:
# Convert '-' into NaN values
clean_companies_df.funding_total_usd.replace('-', np.nan, inplace=True)

# Convert object to numeric float values
clean_companies_df.funding_total_usd = pd.to_numeric(clean_companies_df.loc[:, 'funding_total_usd'])

# Convert NaN into 0
clean_companies_df["funding_total_usd"] = clean_companies_df["funding_total_usd"].fillna(0)

In [10]:
clean_companies_df.head()

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000.0,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000.0,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878.0,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000.0,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,0.0,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


In [11]:
# Convert object to numeric integer values
clean_companies_df.funding_rounds = pd.to_numeric(clean_companies_df.loc[:, 'funding_rounds'])

In [12]:
clean_companies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66338 entries, 0 to 66367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   permalink          66338 non-null  object        
 1   name               66338 non-null  object        
 2   homepage_url       61280 non-null  object        
 3   category_list      63191 non-null  object        
 4   funding_total_usd  66338 non-null  float64       
 5   status             66338 non-null  object        
 6   country_code       59383 non-null  object        
 7   state_code         57794 non-null  object        
 8   region             58311 non-null  object        
 9   city               58313 non-null  object        
 10  funding_rounds     66338 non-null  int64         
 11  founded_at         51122 non-null  object        
 12  first_funding_at   66338 non-null  datetime64[ns]
 13  last_funding_at    66338 non-null  datetime64[ns]
dtypes: dat

#### Drop unncessary columns 

In [13]:
clean_companies_df.drop(columns=['homepage_url'], axis=1, inplace=True)

In [14]:
clean_companies_df.status.value_counts()

operating    53008
closed        6235
acquired      5548
ipo           1547
Name: status, dtype: int64

In [15]:
# See how many companies exited (acquired or IPO'd) more than 3 years ago
clean_companies_df[np.logical_and(
    np.logical_and(clean_companies_df.last_funding_at < pd.to_datetime('2013'),
                   clean_companies_df.first_funding_at > '2005'),
    np.logical_or(clean_companies_df.status == 'ipo',
                   clean_companies_df.status == 'acquired'))]

,permalink,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
20,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,500000.0,acquired,USA,WA,Seattle,Seattle,1,2009-01-01,2009-05-15,2009-05-15
23,/organization/1000memories,1000memories,Curated Web,2535000.0,acquired,USA,CA,SF Bay Area,San Francisco,2,2010-07-01,2010-01-01,2011-02-16
31,/organization/100plus,100Plus,Analytics,1250000.0,acquired,USA,CA,SF Bay Area,San Francisco,2,2011-09-16,2011-11-02,2011-11-30
32,/organization/1010data,1010data,Software,35000000.0,acquired,USA,NY,New York City,New York,1,2000-01-01,2010-03-08,2010-03-08
58,/organization/123people,yelster digital gmbh (former 123people),Local Search|Software|Web Design,0.0,acquired,AUT,9,Vienna,Vienna,1,2007-01-01,2008-01-01,2008-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66316,/organization/zuuka,zuuka!,Apps|Entertainment|Kids|Mobile|Publishing,0.0,acquired,USA,CA,Santa Barbara,Santa Barbara,2,2009-04-10,2011-01-24,2012-06-11
66321,/organization/zvents,Zvents,Concerts|Curated Web|Entertainment|Events|Kids...,55200000.0,acquired,USA,CA,SF Bay Area,San Mateo,4,2005-03-01,2005-10-05,2011-09-30
66336,/organization/zygo-corporation,Zygo Corporation,Manufacturing,9000000.0,acquired,USA,CT,CT - Other,Middlefield,1,NaN,2011-10-24,2011-10-24
66349,/organization/zyncro,Zyncro,Cloud Computing|Enterprise 2.0|Enterprise Soft...,3805520.0,acquired,ESP,56,Barcelona,Barcelona,2,2009-09-01,2011-04-05,2012-02-03


In [17]:
# See how many operating companies were last funded more than 3 years ago
clean_companies_df[np.logical_and(
    np.logical_and(clean_companies_df.last_funding_at < pd.to_datetime('2013'),
                   clean_companies_df.first_funding_at > '2005'),
    clean_companies_df.status == 'operating')]

,permalink,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
3,/organization/0-6-com,0-6.com,Curated Web,2000000.0,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
6,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,Biotechnology,762851.0,operating,CAN,BC,Vancouver,Vancouver,2,1997-01-01,2009-09-11,2009-12-21
12,/organization/1-800-dentist,1-800-DENTIST,Health and Wellness,0.0,operating,USA,CA,Los Angeles,Los Angeles,1,1986-01-01,2010-08-19,2010-08-19
13,/organization/1-800-doctors,1-800-DOCTORS,Health and Wellness,1750000.0,operating,USA,NJ,Newark,Iselin,1,1984-01-01,2011-03-02,2011-03-02
17,/organization/10-20-media,10-20 Media,E-Commerce,2050000.0,operating,USA,MD,Baltimore,Woodbine,4,2001-01-01,2009-06-18,2011-12-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66339,/organization/zylie-the-bear,Zylie the Bear,E-Commerce,75000.0,operating,USA,CT,Hartford,Greenwich,1,2009-01-01,2012-06-07,2012-06-07
66346,/organization/zympi,Zympi,Mobile|Networking|Web Hosting,0.0,operating,USA,CA,Ontario - Inland Empire,Redlands,1,2010-09-03,2011-11-22,2011-11-22
66353,/organization/zyngenia,Zyngenia,Biotechnology,25000000.0,operating,USA,MD,"Washington, D.C.",Gaithersburg,1,2008-01-01,2010-09-09,2010-09-09
66359,/organization/zyrra,Zyrra,E-Commerce,1510500.0,operating,USA,MA,Boston,Cambridge,4,NaN,2010-10-21,2012-10-18


In [16]:
# See how many operating companies were closed more than 3 years ago
clean_companies_df[np.logical_and(
    np.logical_and(clean_companies_df.last_funding_at < pd.to_datetime('2013'),
                   clean_companies_df.first_funding_at > '2005'),
    clean_companies_df.status == 'closed')]

,permalink,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
34,/organization/10bestthings,10BestThings,Curated Web,50000.0,closed,USA,OH,Cleveland,Cleveland,1,2009-04-01,2009-04-01,2009-04-01
47,/organization/11i-solutions,11i Solutions,Enterprise Software,1800000.0,closed,USA,AL,Huntsville,Huntsville,1,NaN,2011-03-22,2011-03-22
76,/organization/169-st,169 ST.,Entertainment|Games,50000.0,closed,USA,FL,Orlando,Lake Mary,1,2009-05-15,2009-06-01,2009-06-01
88,/organization/1bib,1bib,Cars|Curated Web,0.0,closed,CHN,30,Guangzhou,Guangzhou,1,2006-01-01,2008-02-07,2008-02-07
89,/organization/1bog,One Block Off the Grid (1BOG),Clean Technology|Residential Solar,5000000.0,closed,USA,CA,SF Bay Area,San Francisco,1,2008-11-01,2010-02-11,2010-02-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66315,/organization/zuujit,Zuujit,E-Commerce,2700000.0,closed,USA,CA,Santa Barbara,Santa Barbara,1,2010-01-01,2010-04-30,2010-04-30
66318,/organization/zuvvu,Zuvvu,Advertising|Social Media Advertising|Social Me...,12000.0,closed,IND,9,Ahmedabad,Ahmadabad,1,2010-06-01,2011-11-01,2011-11-01
66335,/organization/zygo,Zygo Communications,Mobile|SMS,0.0,closed,GBR,H9,London,London,1,2005-06-21,2007-11-19,2007-11-19
66338,/organization/zykis,Zykis,Automotive|Data Security|Education|Kids|Portal...,800000.0,closed,USA,CA,Anaheim,Irvine,1,2006-10-27,2006-10-27,2006-10-27


In [18]:
# Create dataset containing only non-operting companies that closed or were acquired/IPO'd

current_date = clean_companies_df.last_funding_at.max()

nonoperating_companies_df = clean_companies_df[np.logical_or(
    clean_companies_df.status != 'operating',
    np.logical_and(
        np.logical_and(
            clean_companies_df.status == 'operating',
            clean_companies_df.first_funding_at >
            current_date - pd.offsets.DateOffset(years=10)),
        clean_companies_df.last_funding_at < current_date -
        pd.offsets.DateOffset(years=2, months=6)))].reset_index()

In [19]:
nonoperating_companies_df.drop(columns='index', axis=1, inplace=True)

In [20]:
nonoperating_companies_df.shape

(32426, 13)

In [21]:
# Create binary status column where 0 = closed, 1 = aqcquired/IPO'd
nonoperating_companies_df['status_bool'] = np.nan

# Fill the column
nonoperating_companies_df.loc[np.logical_or(
    nonoperating_companies_df.status == 'closed', nonoperating_companies_df.
    status == 'operating'), 'status_bool'] = 0
nonoperating_companies_df.loc[np.logical_or(
    nonoperating_companies_df.status == 'ipo', nonoperating_companies_df.status
    == 'acquired'), 'status_bool'] = 1

In [22]:
nonoperating_companies_df.status_bool.value_counts()

0.0    25331
1.0     7095
Name: status_bool, dtype: int64

In [23]:
# Number of companies that don't have funding dollar info
(nonoperating_companies_df['funding_total_usd'] == 0.0).sum()

6605

### Round Database

In [31]:
rounds_df.dtypes

company_permalink           object
company_name                object
company_category_list       object
company_country_code        object
company_state_code          object
company_region              object
company_city                object
funding_round_permalink     object
funding_round_type          object
funding_round_code          object
funded_at                   object
raised_amount_usd          float64
dtype: object

In [32]:
rounds_df.shape

(114949, 12)

In [33]:
rounds_df.funding_round_type.unique()

array(['venture', 'seed', 'undisclosed', 'equity_crowdfunding',
       'convertible_note', 'private_equity', 'debt_financing', 'angel',
       'grant', 'secondary_market', 'post_ipo_equity', 'post_ipo_debt',
       'product_crowdfunding', 'non_equity_assistance'], dtype=object)

In [34]:
# Only interested in pre exit and current funding rounds
clean_rounds_df = rounds_df[np.logical_and(
    rounds_df.funding_round_type !=  'post_ipo_equity',
    rounds_df.funding_round_type != 'post_ipo_debt')]

In [35]:
clean_rounds_df.shape

(114159, 12)

In [36]:
clean_rounds_df.funded_at = pd.to_datetime(clean_rounds_df.loc[:, 'funded_at'])

#### Average time between funding rounds

In [37]:
# Sort rounds by company name and dates
clean_rounds_df.sort_values(by=['company_name', 'funded_at'], ascending=True,inplace=True)

<ipython-input-37-3bdc978b4c0e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_rounds_df.sort_values(by=['company_name', 'funded_at'], ascending=True,inplace=True)


In [38]:
# Create column for time between funding rounds
clean_rounds_df['time_between_next_round'] = clean_rounds_df.groupby('company_name').funded_at.diff()

<ipython-input-38-5a4c37da082f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_rounds_df['time_between_next_round'] = clean_rounds_df.groupby('company_name').funded_at.diff()


In [39]:
# Calculate average time between rounds and average amount raised
average_round_time_amount = clean_rounds_df.groupby('company_name').agg({
    'time_between_next_round': pd.Series.mean,
    'raised_amount_usd' : 'mean'
}).rename(columns = {
         'time_between_next_round' : 'avg_time_between_rounds',
         'raised_amount_usd' : 'avg_raised_usd'
          }).reset_index()

In [46]:
# Merge into nonoperating companies df
nonoperating_companies_df = nonoperating_companies_df.merge(
                             average_round_time_amount, how='left', left_on='name', right_on='company_name')

In [45]:
average_round_time_amount

,company_name,avg_time_between_rounds,avg_raised_usd
0,"#BratPackStyle, LLC.",NaT,NaN
1,#HASHOFF,246 days,477500.0
2,#fame,NaT,10000000.0
3,#waywire,NaT,1750000.0
4,&TV Communications,111 days,2000000.0
...,...,...,...
65832,İş kutusu video ajansı,NaT,100000.0
65833,Еnso.fm,NaT,27838.0
65834,​Shwrüm,NaT,50000.0
65835,​WestEd,NaT,3000000.0


In [49]:
clean_rounds_df.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,time_between_next_round
14800,/organization/bratpackstyle-llc,"#BratPackStyle, LLC.",Content Discovery|E-Commerce|Fashion|Lifestyle...,USA,NY,New York City,New York,/funding-round/7f52613bbda1edcaed3844d806d62757,seed,NaN,2015-06-01,NaN,NaT
43095,/organization/hashoff,#HASHOFF,Digital Media|Internet|Social Media,USA,CO,Denver,Denver,/funding-round/669d6203c0374e6cf0e8d10f75ba0b8a,debt_financing,NaN,2014-12-08,455000.0,NaT
43096,/organization/hashoff,#HASHOFF,Digital Media|Internet|Social Media,USA,CO,Denver,Denver,/funding-round/889482c82e762d06c02a1dfa8fb41ccf,debt_financing,NaN,2015-08-11,500000.0,246 days
0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,NaT
109523,/organization/waywire,#waywire,Entertainment|News|Politics|Social Media,USA,NY,New York City,New York,/funding-round/cc409188fa2b63482bd9008f682c2efa,seed,NaN,2012-06-30,1750000.0,NaT


#### Time between first and second rounds

In [67]:
# Create a column in nonoperating companies df
nonoperating_companies_df['time_between_first_rounds'] = dt.timedelta(0)

for company in clean_rounds_df.company_name.unique():
    if len(clean_rounds_df[clean_rounds_df.company_name ==
                           company].funded_at) > 1:
        nonoperating_companies_df.loc[
            nonoperating_companies_df.name ==
            company, 'time_bw_first_rounds'] = clean_rounds_df[
                clean_rounds_df.company_name == company].funded_at.values[
                    1] - clean_rounds_df[clean_rounds_df.company_name ==
                                         company].funded_at.values[0]



In [69]:
nonoperating_companies_df.tail()

,permalink,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,...,last_funding_at,status_bool,company_name_x,avg_time_between_rounds_x,avg_raised_usd_x,company_name_y,avg_time_between_rounds_y,avg_raised_usd_y,time_bw_first_rounds,time_between_first_rounds
32421,/organization/zyraz-technology,Zyraz Technology,Biotechnology,15419877.0,closed,MYS,14,MYS - Other,Damansara New Village,4,...,2013-02-15,0.0,Zyraz Technology,624 days 00:00:00,3854969.25,Zyraz Technology,624 days 00:00:00,3854969.25,619 days,0 days
32422,/organization/zyrra,Zyrra,E-Commerce,1510500.0,operating,USA,MA,Boston,Cambridge,4,...,2012-10-18,0.0,Zyrra,242 days 16:00:00,377625.00,Zyrra,242 days 16:00:00,377625.00,25 days,0 days
32423,/organization/zystor,Zystor,Biotechnology,8500000.0,acquired,USA,WI,Milwaukee,Milwaukee,1,...,2004-12-16,1.0,Zystor,NaT,8500000.00,Zystor,NaT,8500000.00,NaT,0 days
32424,/organization/zytoprotec,Zytoprotec,Biotechnology,2686600.0,operating,AUT,3,Vienna,Gerasdorf Bei Wien,1,...,2013-01-29,0.0,Zytoprotec,NaT,2686600.00,Zytoprotec,NaT,2686600.00,NaT,0 days
32425,/organization/zznode-science-and-technology-co...,ZZNode Science and Technology,Enterprise Software,1587301.0,operating,CHN,22,Beijing,Beijing,1,...,2012-04-01,0.0,ZZNode Science and Technology,NaT,1587301.00,ZZNode Science and Technology,NaT,1587301.00,NaT,0 days
